In [1]:
#import
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [2]:
cus = pd.read_csv("open/cus_info.csv")
iem = pd.read_csv("open/iem_info.csv")
hist = pd.read_csv("open/stk_bnc_hist.csv")
train = pd.read_csv("open/stk_hld_train.csv")
test = pd.read_csv("open/stk_hld_test.csv")

submission = pd.read_csv("open/sample_submission.csv")

train["hist_d"] = train["hold_d"]*0.45
train.hist_d = np.trunc(train["hist_d"])

train.head()

,act_id,iem_cd,byn_dt,hold_d,hist_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,4.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,36.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,2.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,9.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,145.0


In [3]:
train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])
train_data.head()

,act_id,iem_cd,byn_dt,hold_d,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,4.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,36.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,2.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,9.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,145.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99


In [4]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.
train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

train_data.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,4.0,1,9,3,2,2,9,5,5,8,101,1,1,1,274.0,11782000.0,5000.0,43000.0
1,36.0,1,9,3,2,2,9,5,5,8,1361,9,1,1,2.0,4990000.0,5000.0,2495000.0
2,2.0,1,9,3,2,2,9,5,5,8,2530,12,2,99,786.0,14619600.0,1000.0,18600.0
3,9.0,1,9,3,2,2,9,5,5,8,1969,8,2,1,60.0,462000.0,500.0,7700.0
4,145.0,1,9,3,2,2,9,5,5,8,1696,10,3,99,183.0,8125200.0,5000.0,44400.0


In [5]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [6]:
input_var = list(train_data)

X_features = train_data[input_var]
y_target = train_label

input_var

['hist_d',
 'sex_dit_cd',
 'cus_age_stn_cd',
 'ivs_icn_cd',
 'cus_aet_stn_cd',
 'mrz_pdt_tp_sgm_cd',
 'lsg_sgm_cd',
 'tco_cus_grd_cd',
 'tot_ivs_te_sgm_cd',
 'mrz_btp_dit_cd',
 'iem_krl_nm',
 'btp_cfc_cd',
 'mkt_pr_tal_scl_tp_cd',
 'stk_dit_cd',
 'bnc_qty',
 'tot_aet_amt',
 'stk_par_pr',
 'stk_p']

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size=0.2, random_state=156)

In [8]:
from sklearn.model_selection import GridSearchCV

def get_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_train, y_train)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('최적 평균 RMSE 값:', np.round(rmse, 4))
    print('최적 파라미터:', grid_model.best_params_)
    
    return grid_model.best_estimator_

In [9]:
import xgboost as xgb

# xgb_params 값을 바꾸어주며 learning_rate, max_depth 등 파라미터 성능 비교 시도했었음.
xgb_params = {'n_estimators':[3300]}

xgb_reg = xgb.XGBRegressor(colsample_bytree=0.8, max_depth=5, learning_rate= 0.1, n_estimators=3300,
                           random_state =7, nthread = -1, n_jobs=-1)

best_xgb = get_best_params(xgb_reg, xgb_params)

최적 평균 RMSE 값: 0.6352
최적 파라미터: {'n_estimators': 3300}


In [10]:
best_xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3300, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [11]:
from lightgbm import LGBMRegressor

# lgbm_params 값을 바꾸어주며 learning_rate, max_depth 등 파라미터 성능 비교 시도했었음.
lgbm_params = {'n_estimators':[3300]}

lgbm_reg = LGBMRegressor(n_estimators=3300, bagging_fraction=0.7, learning_rate=0.1,
                         max_depth=4, subsample=0.7, feature_fraction=0.9, boosting_type='gbdt',
                         colsample_bytree=0.5, reg_lambda=5, n_jobs=-1)

best_lgbm = get_best_params(lgbm_reg, lgbm_params)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fractio

In [12]:
best_lgbm

LGBMRegressor(bagging_fraction=0.7, colsample_bytree=0.5, feature_fraction=0.9,
              max_depth=4, n_estimators=3300, reg_lambda=5, subsample=0.7)

In [13]:
X_train = train_data[input_var]
X_test = test_data[input_var]

y_train = train_label

In [14]:
best_xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3300, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [36]:
import xgboost as xgb

# 위의 베스트 파라미터 사용
xgb_reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=3300, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [43]:
xgb_reg.fit(X_train, y_train)

xgb_pred = np.expm1(xgb_reg.predict(X_test))

C:\Users\user\AppData\Local\Temp/ipykernel_24772/3403767322.py:3: RuntimeWarning: overflow encountered in expm1
  xgb_pred = np.expm1(xgb_reg.predict(X_test))


In [44]:
best_lgbm

LGBMRegressor(bagging_fraction=0.7, colsample_bytree=0.5, feature_fraction=0.9,
              max_depth=4, n_estimators=3300, reg_lambda=5, subsample=0.7)

In [45]:
from lightgbm import LGBMRegressor

# 위의 베스트 파라미터 사용
lgbm_reg = LGBMRegressor(bagging_fraction=0.7, colsample_bytree=0.5, feature_fraction=0.9,
              max_depth=4, n_estimators=3300, reg_lambda=5, subsample=0.7)

In [46]:
lgbm_reg.fit(X_train, y_train)

lgbm_pred = np.expm1(lgbm_reg.predict(X_test.values))

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7


C:\Users\user\AppData\Local\Temp/ipykernel_24772/2765338935.py:3: RuntimeWarning: overflow encountered in expm1
  lgbm_pred = np.expm1(lgbm_reg.predict(X_test.values))


In [52]:
ensemble = xgb_pred*0.3 + lgbm_pred*0.7 # 테스트 결과 이 비율이 성능이 높았음.

In [57]:
test_data['hold_d'] = ensemble

In [58]:
test_data[['hold_d']].head(5)

,hold_d
0,inf
1,inf
2,inf
3,inf
4,1.182425e+09


In [59]:
test_data[['hold_d']].to_csv("submission.csv", index=False)

In [60]:
test_data.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,hold_d
0,153,1,9,3,2,2,9,5,5,8,418,4,1,1,300.0,3945000.0,5000.0,13150.0,inf
1,335,1,9,3,2,2,9,5,5,8,2230,10,3,99,198.0,2524500.0,500.0,12750.0,inf
2,139,1,9,3,2,2,9,5,5,8,1515,13,2,99,138.0,4291800.0,500.0,31100.0,inf
3,236,1,9,3,2,2,9,5,5,8,2681,13,3,99,2972.0,6716720.0,500.0,2260.0,inf
4,9,1,9,3,2,2,9,5,5,8,2450,13,1,2,10.0,464500.0,100.0,46450.0,1.182425e+09


models = []

folds = KFold(n_splits=10)

for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

    
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)
predict

submission["hold_d"] = np.round(predict)

submission.loc[submission.hold_d-test.hist_d>146,'hold_d']=test.hist_d+146
submission.to_csv("dacon_baseline1.csv", index = False)